In [ ]:
import re

In [ ]:
old = R"""
#include "pybind11/pybind11.h"

#include "drake/bindings/pydrake/documentation_pybind.h"
#include "drake/bindings/pydrake/pydrake_pybind.h"
#include "drake/multibody/parsers/package_map.h"

#define D(...) DOC(drake, multibody, parsing, __VA_ARGS__)
#define D(...) DOC(__VA_ARGS__)

namespace drake {
namespace pydrake {

PYBIND11_MODULE(parsers, m) {
  using drake::multibody::parsing::PackageMap;

  m.doc() = "Tools for loading robots from various files";

  py::class_<PackageMap>(m, "PackageMap", D(PackageMap))
    .def(py::init<>(), D(PackageMap, PackageMap))
    .def("Add", &PackageMap::Add, D(PackageMap, Add))
    .def("Contains", &PackageMap::Contains, D(PackageMap, Contains))
    .def("size", &PackageMap::size, D(PackageMap, size))
    .def("GetPath", &PackageMap::GetPath, D(PackageMap, GetPath))
    .def("PopulateFromFolder", &PackageMap::PopulateFromFolder,
         D(PackageMap, PopulateFromFolder))
    .def("PopulateFromEnvironment", &PackageMap::PopulateFromEnvironment,
         D(PackageMap, PopulateFromEnvironment))
    .def("PopulateUpstreamToDrake", &PackageMap::PopulateUpstreamToDrake,
         D(PackageMap, PopulateUpstreamToDrake));
}

}  // namespace pydrake
}  // namespace drake
"""

In [ ]:
define_pattern = r"#define D\(...\) DOC\((.*?)__VA_ARGS__\)"

def define_sub(m):
    pieces = ["pydrake_doc"] + m.group(1).split(", ")
    pieces.remove('')
    return "auto& doc = " + ".".join(pieces) + ";"

usage_pattern = r"\bD\((.*?)\)"

def usage_sub(m):
    pieces = m.group(1).split(", ")
    suffix = ""
    try:
        index = int(pieces[-1])
        pieces = pieces[:-1]
        suffix = "_{}".format(index)
    except ValueError:
        pass
    if len(pieces) > 1 and pieces[-1] == pieces[-2]:
        pieces[-1] = "ctor"
    return "doc." + ".".join(pieces) + ".doc" + suffix

new = old
new = re.sub(define_pattern, define_sub, new, re.MULTILINE | re.DOTALL)
new = re.sub(usage_pattern, usage_sub, new, re.MULTILINE | re.DOTALL)

print(new)